# testscript.ipynb

A Jupyter notebook for testing and development of the AmBIENCe2ABM module.

In [ ]:
## Import modules

import pandas as pd
import numpy as np
from itertools import product
import ambience2abm as amb

In [ ]:
## Read the raw data and assumptions.

ambience = amb.AmBIENCeDataset(
    interior_node_depth=0.1,
    period_of_variations=2225140,
)
ambience.data

In [ ]:
## Create unique building periods

building_periods = ambience.building_periods()
building_periods

In [ ]:
## Check the building type to stock mappings.

ambience.building_type_mappings

In [ ]:
## Check the building stock statistics

ambience.building_stock_statistics

In [ ]:
## Check structure type assumptions

ambience.structure_types

In [ ]:
## Calculate the structure statistics

ambience.calculate_structure_statistics()

In [ ]:
## Check fenestration assumptions.

ambience.fenestration

In [ ]:
## Check ventilation assumptions.

ambience.ventilation

In [ ]:
## Calculate ventilation and fenestration statistics

ambience.calculate_ventilation_and_fenestration_statistics()